In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
def loadData():
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/31_DM/20_homeWork2/data/SeoulBikeData.csv')
    # Index(['Date', 'Rented Bike Count', 'Hour', 'Temperature', 'Humidity(%)',
    #        'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature',
    #        'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
    #        'Holiday', 'Functioning Day'],dtype='object')

    # Index(['Spring', 'Summer', 'Autumn', 'Winter'], dtype='object')
    # Index(['No Holiday', 'Holiday'], dtype='object')
    # Index(['Yes', 'No'], dtype='object')
    # print(df['Seasons'].value_counts().index)
    # print(df['Holiday'].value_counts().index)
    # print(df['Functioning Day'].value_counts().index)

    #  drop column of Date
    df.drop("Date", axis=1, inplace=True)

    # No Holiday 編碼 0
    #    Holiday 編碼 1
    df.loc()[df.loc[:, 'Holiday'] == 'No Holiday', 'Holiday'] = 0
    df.loc()[df.loc[:, 'Holiday'] == 'Holiday', 'Holiday'] = 1
    df['Holiday'] = df['Holiday'].astype(int)
    # No  編碼 0
    # Yes 編碼 1
    df.loc[df.loc[:, 'Functioning Day'] == 'No', 'Functioning Day'] = 0
    df.loc[df.loc[:, 'Functioning Day'] == 'Yes', 'Functioning Day'] = 1
    df['Functioning Day'] = df['Functioning Day'].astype(int)

    # delete 'Rented Bike Count' of 0 ， MAPE!=0
    # delete 295 of row
    print(df['Rented Bike Count'][df['Rented Bike Count'] == 0].count())
    df = df[df['Rented Bike Count'] != 0]

    #  one hat
    df['Hour'] = df['Hour'].astype(str)
    df1 = pd.get_dummies(df)
    return df1

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))
    return mape

def _score(y, y_hat):
    y = np.array(y)
    df = pd.DataFrame(
        [[r2_score(y, y_hat),
          mean_absolute_error(y, y_hat),
          mean_squared_error(y, y_hat, squared=False),
          MAPE(y,y_hat)
          ]],
        columns=[
            'r2_score',
            'MAE',
            'RMSE',
            'MAPE'
        ]
    )
    return df

def train_Regressor(estimator_list: list, X_train, Y_train, **kwargs):
    for estimator in estimator_list:
        estimator.fit(X_train, Y_train)
    return estimator_list

In [3]:
df = loadData()

Y = df[['Rented Bike Count']]
X = df.drop(Y.columns, axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=321)

295


In [4]:
df

,Rented Bike Count,Temperature,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature,Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Hour_0,Hour_1,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_2,Hour_20,Hour_21,Hour_22,Hour_23,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,Seasons_Autumn,Seasons_Spring,Seasons_Summer,Seasons_Winter
0,254,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,204,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,173,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,107,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,78,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1003,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8756,764,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8757,694,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
8758,712,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [5]:
X_train

,Temperature,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature,Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Hour_0,Hour_1,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_2,Hour_20,Hour_21,Hour_22,Hour_23,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,Seasons_Autumn,Seasons_Spring,Seasons_Summer,Seasons_Winter
5726,34.1,57,1.8,667,24.3,1.13,0.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1876,-5.2,28,2.5,2000,-20.9,0.00,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
516,4.5,56,0.8,836,-3.5,0.71,0.0,0.0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7786,13.4,46,0.8,1718,1.9,1.61,0.0,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6117,28.9,83,0.3,1449,25.7,0.00,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,11.9,57,1.0,1793,3.6,0.00,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8655,13.2,43,1.1,714,0.8,1.26,0.0,0.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5977,28.2,68,0.9,2000,21.7,0.00,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4268,20.6,41,2.3,1788,6.8,0.09,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
Y_train

,Rented Bike Count
5726,507
1876,20
516,301
7786,936
6117,1337
...,...
3506,409
8655,794
5977,619
4268,2044


In [7]:
def normalization_X_Y(X_train, Y_train, X_test, Y_test, **kwargs):
    from sklearn.preprocessing import MinMaxScaler
    scaler_X = MinMaxScaler()
    scaler_Y = MinMaxScaler()
    X_train_temp = scaler_X.fit_transform(X_train)
    X_test_temp = scaler_X.transform(X_test)
    Y_train_temp = scaler_Y.fit_transform(Y_train)
    Y_test_temp = scaler_Y.transform(Y_test)
    parmer_dict = {"X_train": X_train_temp,
                   "Y_train": Y_train_temp,
                   "X_test": X_test_temp,
                   "Y_test": Y_test_temp,
                   "scaler_X": scaler_X,
                   "scaler_Y": scaler_Y
                   }
    return parmer_dict

In [8]:
# {'svm_parameter': {'degree': 3, 'coef0': 0.5427121056602934, 'C': 2.232528796987344, 'gamma': 0.07478899690842146, 'epsilon': 0.053347237333679826}}
svm_parameter={'degree': 3, 'coef0': 0.5427121056602934, 'C': 2.232528796987344, 'gamma': 0.07478899690842146, 'epsilon': 0.053347237333679826}
estimator_list = [
    KNeighborsRegressor(),
    SVR(),
    RandomForestRegressor(),
    XGBRegressor()
]
prmer_dict = normalization_X_Y(X_train, Y_train, X_test, Y_test)

estimator_list = train_Regressor(estimator_list, prmer_dict['X_train'], prmer_dict['Y_train'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


[10:24:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [9]:
def test_score(estimator_list: list, prmer_dict: dict, **kwargs):
    train_score_list = []
    test_score_list = []
    for estimator in estimator_list:
        Y_train_pred = estimator.predict(prmer_dict["X_train"])
        Y_train_inverse = prmer_dict['scaler_Y'].inverse_transform(Y_train_pred.reshape(-1, 1))
        Y_train = prmer_dict['scaler_Y'].inverse_transform(prmer_dict['Y_train'].reshape(-1, 1))

        Y_test_pred = estimator.predict(prmer_dict['X_test'])
        Y_test_inverse = prmer_dict['scaler_Y'].inverse_transform(Y_test_pred.reshape(-1, 1))
        Y_test = prmer_dict['scaler_Y'].inverse_transform(prmer_dict['Y_test'].reshape(-1, 1))

        df_train_score = _score(Y_train, Y_train_inverse)
        df_train_score.index = [estimator.__class__.__name__]
        df_test_score = _score(Y_test, Y_test_inverse)
        df_test_score.index = [estimator.__class__.__name__]
        train_score_list.append(df_train_score)
        test_score_list.append(df_test_score)
    return pd.concat(train_score_list), pd.concat(test_score_list)
df_train_score, df_test_score = test_score(estimator_list, prmer_dict)

In [10]:
df_train_score

,r2_score,MAE,RMSE,MAPE
KNeighborsRegressor,0.843907,157.874808,252.896962,0.926570
SVR,0.824378,201.161260,268.250837,1.069419
RandomForestRegressor,0.979886,55.900427,90.783079,0.209077
XGBRegressor,0.836950,181.321233,258.470995,0.744532


In [11]:
df_test_score

,r2_score,MAE,RMSE,MAPE
KNeighborsRegressor,0.782911,192.204253,303.310294,1.161702
SVR,0.815380,209.680298,279.709667,1.126560
RandomForestRegressor,0.853531,155.940021,249.138500,0.513829
XGBRegressor,0.823676,188.986619,273.353406,0.743210


# KNN 不同的K值

In [ ]:
def knn_to_K(K:int,prmer_dict: dict):
  test_score_list = []
  for k in range(1,K):
    print(f"-----{k} start------")
    knn=KNeighborsRegressor(n_neighbors=k)
    knn.fit(prmer_dict["X_train"],prmer_dict["Y_train"])

    Y_test_pred = knn.predict(prmer_dict['X_test'])
    Y_test_inverse = prmer_dict['scaler_Y'].inverse_transform(Y_test_pred.reshape(-1, 1))
    Y_test = prmer_dict['scaler_Y'].inverse_transform(prmer_dict['Y_test'].reshape(-1, 1))
    df_test_score = _score(Y_test, Y_test_inverse)
    df_test_score.index = [k]
    test_score_list.append(df_test_score)
    print(f"-----{k} end  ------")
  return pd.concat(test_score_list)

df_knn=knn_to_K(15,prmer_dict)


In [17]:
df_knn

,r2_score,MAE,RMSE,MAPE
1,0.684739,221.813349,365.513743,1.122384
2,0.746091,202.826344,328.025603,1.188497
3,0.778159,194.993503,306.612374,1.145441
4,0.783701,192.661548,302.758113,1.141856
5,0.782911,192.204253,303.310294,1.161702
6,0.788367,193.176413,299.474581,1.167283
7,0.778800,196.111552,306.168650,1.220853
8,0.776633,197.543709,307.664851,1.254950
9,0.773736,199.296581,309.653575,1.299341
10,0.770519,201.172711,311.846855,1.338665
